In [ ]:
import sys
sys.path.append('../library')
sys.path.append('../library/rtde')
import logging
import rtde.rtde as rtde
import rtde.rtde_config as rtde_config
import time
import math
import subprocess
import socket 
import os
from quaternion import quaternion
from transform_coordinate import transform_coordinate
import keyboard
from IPython.display import clear_output
from itertools import cycle
import numpy

# 0 1 2 3 4 5 6 7  8  9  10   11  12  13  14  15   16    17   18 
# b b b b b b b A  B  s  rup  rp  rm  x    y  s    lup   lp   lm 
#rup:righ up  rp:right point rm:right mosht

In [ ]:
#logging.basicConfig(level=logging.INFO)

ROBOT_HOST = '192.168.253.1'
ROBOT_PORT = 30004
config_filename = '../config/pose_pilot_configuration_with_button.xml'

UDP_IP = "127.0.0.1"
UDP_PORT = 5005
sock = socket.socket(socket.AF_INET, # Internet
                     socket.SOCK_DGRAM) # UDP
#sock.setblocking(0)
sock.bind((UDP_IP, UDP_PORT))


keep_running = True
#logging.getLogger().setLevel(logging.INFO)

In [ ]:
conf = rtde_config.ConfigFile(config_filename)
state_names, state_types = conf.get_recipe('state')
setp_names, setp_types = conf.get_recipe('setp')
watchdog_names, watchdog_types = conf.get_recipe('watchdog')
button_names, button_types = conf.get_recipe('button')

In [ ]:
con = rtde.RTDE(ROBOT_HOST, ROBOT_PORT)
con.connect()

# get controller version
con.get_controller_version()

# setup recipes
con.send_output_setup(state_names, state_types)
setp = con.send_input_setup(setp_names, setp_types)
watchdog = con.send_input_setup(watchdog_names, watchdog_types)
button = con.send_input_setup(button_names, button_types)

In [ ]:
def setp_to_list(setp):
    list = []
    for i in range(0,6):
        list.append(setp.__dict__["input_double_register_%i" % i])
    return list

def list_to_setp(setp, list):
    for i in range (0,6):
        setp.__dict__["input_double_register_%i" % i] = list[i]
    return setp

def button_to_list(button):
    list = []
    for i in range(0,2):
        list.append(button.__dict__["input_int_register_%i" % (i+1)])
    return list

def list_to_button(button, list):
    for i in range (0,2):
        button.__dict__["input_int_register_%i" % (i+1)] = list[i]
    return button

In [ ]:
# Initialization
# setp
setp.input_double_register_0 = 0
setp.input_double_register_1 = 0
setp.input_double_register_2 = 0
setp.input_double_register_3 = 0
setp.input_double_register_4 = 0
setp.input_double_register_5 = 0

# button
button.input_int_register_1 = 0
button.input_int_register_2 = 0
  
# The function "rtde_set_watchdog" in the "rtde_control_loop.urp" creates a 1 Hz watchdog
watchdog.input_int_register_0 = 0

# controller to robot gripper transformation
tr_con_rob_rot_vec_aangle = [0, math.pi, 0]
tr_con_rob_rot_vec = quaternion.axis_angle_to_quaternion(tr_con_rob_rot_vec_aangle)
tr_con_rob_cor = transform_coordinate([0, 0, 0], quaternion(1, 0, 0, 0), tr_con_rob_rot_vec)

# Button state
tracking = False
last_tracking = False
last_index_trigger = False
last_left_fist_button = False
on_surface = False
S_button_last = False
caution = False
distance_far = False
free_drive = False
last_robot_awake = False
total_it_number_delay = 10
last_reach_after_maneuver = False
last_go_normal = False
i_delay = total_it_number_delay



#saving initials
auto =0
save = 0
saved_traj = []
saved_traj_from_rob = []
traj_i = int(0)
fd = 0

new_setp_normal_tracking=[]
#robot posisition saving initials
rpi = 0
on_off =  cycle([1,0])
off_on = cycle([0,1])

# Force
contact_force_threshold = 1000.
rob_in_contact = False
last_rob_in_contact = False

# Trajectory files
traj_dir = "../trajectories/circle/"
default_traj_file_name = "circle_new"

# auto
traj_v_preset = 2
speed_up = 2
initial_auto_segment_index_length = 10
last_auto_segment_index_length = 10
max_auto_force = 40

In [ ]:
## Load last trajectory file name and trajectory
if traj_dir[-1] != '/':
    traj_dir = traj_dir + '/'
traj_dir_name = traj_dir.split('/')[-2]
traj_dir_file_names = os.listdir(traj_dir)
traj_dir_file_names.remove(default_traj_file_name)
if len(traj_dir_file_names) > 0:
    last_file_name = traj_dir_file_names[-1]
else:
    last_file_name = ""

if last_file_name != "":
    last_file_num = int(last_file_name.split(traj_dir_name)[-1])
else: 
    last_file_num = 0

# Load trajectory
if default_traj_file_name != "":
    load_traj_file_name = default_traj_file_name
else:
    load_traj_file_name = last_file_name

if load_traj_file_name != "":
    load_traj_file_path = traj_dir + load_traj_file_name
    with open(load_traj_file_path) as f:
        content = f.readlines()
    saved_traj = [[float(point) for point in line.split(',')] for line in content]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
x = np.array([t[0] for t in saved_traj])

z = np.array([t[2] for t in saved_traj])
l = plt.plot(x, z, 1, 1)
plt.show()

In [ ]:
#start data synchronization
if not con.send_start():
    sys.exit()

# control loop
while keep_running:
    # receive the current state
    state = con.receive()
    
    # If connection is broken break
    if state is None:
        break;
    
    # Send data to Unity
    net_torque = (state.actual_TCP_force[3]**2 + state.actual_TCP_force[4]**2 + state.actual_TCP_force[5]**2)**0.5
    sock.sendto((str(state.tcp_force_scalar)+","+str(net_torque)+","+str(auto)+","+str(77)+","+str(len(saved_traj))+","+str(traj_i)), ("127.0.0.1", 11000))
    
    # Receieve data from Unity
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    controller_state = [float(s) for s in data.split(',')]

    # to right hand
    controller_state[2] = -controller_state[2]
    controller_state[5] = -controller_state[5]
    controller_state[6] = -controller_state[6]*math.pi/180.0
        

    # are we in tracking mode?
    index_trigger = controller_state[11] > 0
    
    if ((not last_index_trigger) & index_trigger) :
        tracking =  not tracking
    
    
    # Is robot in contact?
    rob_in_contact =  state.tcp_force_scalar >= contact_force_threshold
    if rob_in_contact and not last_rob_in_contact:
                    contact_z = state.actual_TCP_pose[2]
                    #print(contact_z)

    #(1)check if robot has just became awake. (2)if so will put "tracking" on "False"
    #(1)
    robot_awake = state.runtime_state == 2
    if robot_awake & (not last_robot_awake):
        is_beginning = True
    else:
        is_beginning = False
    #(2)
    if is_beginning:
        tracking = False
        if auto == 1:
            auto = 0
            traj_i = len(saved_traj) + 1
            #caution = True
             
        
            
    # get new robot set point
    if tracking:
        if not last_tracking:
            # set current controller loc-ori to current rob loc-ori
            # get controller coordinates transformations
            con_base_loc = controller_state[0:3]
            con_base_ori_aangle = [i*controller_state[6] for i in controller_state[3:6]]
            con_base_ori = quaternion.axis_angle_to_quaternion(con_base_ori_aangle)

            tr_con_cor = transform_coordinate(con_base_loc, con_base_ori, con_base_ori) 
            
            # get robot coordinates transformations
            rob_base_loc = state.actual_TCP_pose[0:3]
            rob_base_ori_aangle = state.actual_TCP_pose[3:6]
            rob_base_ori = quaternion.axis_angle_to_quaternion(rob_base_ori_aangle)

            tr_rob_cor = transform_coordinate(rob_base_loc, rob_base_ori, rob_base_ori)
            new_setp = state.actual_TCP_pose
        else:
            # Controller Current Orientation in Controller cordinates
            con_loc = controller_state[0:3]
            con_ori_aangle = [i*controller_state[6] for i in controller_state[3:6]]
            con_ori = quaternion.axis_angle_to_quaternion(con_ori_aangle)

            rel_con_loc, rel_con_ori = tr_con_cor.to_new(con_loc, con_ori)
            rel_rob_loc, rel_rob_ori = tr_con_rob_cor.to_new(rel_con_loc, rel_con_ori)
            rob_loc, rob_ori = tr_rob_cor.to_old(rel_rob_loc, rel_rob_ori)

            # Z correction for contact
            if rob_in_contact and rob_loc[2] <= contact_z:
                rob_loc[2] = contact_z
            if auto == 0:
                new_setp = rob_loc + rob_ori.to_axis_angle()
                new_setp_normal_tracking = new_setp
    else:
        new_setp = state.actual_TCP_pose
        
    # y constant surface with constant oriantation
    S_button = controller_state[15]>0
    if S_button & (not S_button_last) :
        on_surface = not on_surface  
        #caution = not on_surface
        last_for_surface_setp = new_setp
        
    if on_surface:
        new_setp[1] = last_for_surface_setp[1]
        new_setp[3:] = last_for_surface_setp[3:]
    
        
    #Saving
    #get button x
    if controller_state[13] > 0 and not x_button_last :
        save = 1
        saved_traj = []
    #get button y
    if controller_state[14] > 0 and save == 1:
        save=0
        if saved_traj != []:
            last_file_num = last_file_num + 1
            save_file_path = traj_dir + traj_dir_name + str(last_file_num)
            with open(save_file_path, 'w') as f:
                for pose in saved_traj:
                    f.write(",".join([str(point) for point in pose]) + '\n')
                    
                    
    if save == 1:
         saved_traj.append(state.actual_TCP_pose)
         #saved_traj_from_rob.append(state.actual_TCP_pose)
    
    #Auto
     #get button B
    if controller_state[8] > 0 and auto == 0 and save == 0 and len(saved_traj) > 0:
        auto = 1
        traj_i = int(0)
        xx = new_setp[0] - saved_traj[0][0]  
        yy = new_setp[1] - saved_traj[0][1]
       # print(new_setp)

    print(state.runtime_state)    
    print("force: " + str(state.tcp_force_scalar) + "torque: " + str(net_torque))
    print(distance_far)
    if auto == 1:
        if (traj_i != 1) | (not distance_far):  # traj_i=0 passes to get the first point, then it doesn't go through until distance_far is false
            new_setp = saved_traj[traj_i]
            new_setp[0] = new_setp[0] #+ xx
            new_setp[1] = new_setp[1] #+ yy
            traj_v = traj_v_preset
            traj_i = traj_i + 1
#             if traj_i <= initial_auto_segment_index_length or traj_i >= (len(saved_traj) - last_auto_segment_index_length):
#                 traj_v = 1
#             if controller_state[10] > 0:
#                 #traj_i = traj_i + speed_up * traj_v - int(state.tcp_force_scalar / (max_auto_force / (traj_v * speed_up)))
#                 traj_i = traj_i + speed_up * traj_v
#             else:
#                 #traj_i = traj_i + traj_v - int(state.tcp_force_scalar / (max_auto_force / traj_v))
#                 traj_i = traj_i + traj_v
            
            
            hit_da_potato = state.tcp_force_scalar >= max_auto_force
            if traj_i >= len(saved_traj) or hit_da_potato:
                auto = 0
                # caution = True
    begin_auto = traj_i == 1
    
    
    
    #Determining the two digit for buttons, [free_drive, distance_far]

    # Get button left fist and turn free drive on/off
    left_fist_button = controller_state[18] > 0
    if left_fist_button  & (not last_left_fist_button) :    #turn on/off free drive
        free_drive = not free_drive
        
    #Check if the distance is almost zero for going as normal tracking
    go_normal = new_setp_normal_tracking == new_setp
    reach_after_maneuver = go_normal and (not last_go_normal)
    reach_before_maneuver = (not go_normal) and last_go_normal
    
    if reach_after_maneuver or reach_before_maneuver:
        caution = True
    #reach = reach_after_maneuver | reach_before_maneuver
    #if reach :
    if caution:
        now_controller = numpy.array(new_setp[0:3])
        now_robot = numpy.array(state.actual_TCP_pose[0:3])
        distance = numpy.linalg.norm(now_controller-now_robot)
        if distance < 0.01:
            caution = False
            distance_far = False
        else:
            distance_far = True

                
        
    if reach_after_maneuver & (not last_reach_after_maneuver):
        i_delay = 0
    if i_delay < total_it_number_delay:
        free_drive = True
        i_delay = i_delay+1
        print("oppppppppppppppppppppppp")
        print(i_delay)
        
    print("left_fist   "+str(left_fist_button)) 
    print("last_left fist   "+str(last_left_fist_button))
        


    list_to_setp(setp, new_setp)
    list_to_button(button, [int(free_drive),int(distance_far)])
    print(button, [int(free_drive),int(distance_far)])
    
    # Send button data
    con.send(button)
    
    # send new setpoint        
    con.send(setp)
    
    # kick watchdog
    con.send(watchdog)
    
    #save is as last moment
    last_left_fist_button = controller_state[18] > 0
    S_button_last = controller_state[15] > 0
    x_button_last = controller_state[13] > 0
    last_robot_awake =  state.runtime_state == 2
    last_reach_after_maneuver = reach_after_maneuver
    last_go_normal = go_normal
    last_index_trigger = index_trigger
    last_rob_in_contact = rob_in_contact
    last_tracking = tracking
    

In [ ]:
con.send_pause()
con.disconnect()
sys.stdout.flush()